import pandas as pd

In [1]:
import pandas as pd

In [2]:
vino = pd.read_csv(r"C:\Users\Dell\Desktop\Restaurant\Vino Cultura.csv", error_bad_lines=False);
almond = pd.read_csv(r"C:\Users\Dell\Desktop\Hotel\Almond Business Hotel.csv", error_bad_lines=False);
brasserie = pd.read_csv(r"C:\Users\Dell\Desktop\Restaurant\Brasserie Au Bon Plaisir.csv", error_bad_lines=False);


tripadv_review = pd.concat([vino, almond, brasserie])
tripadv_review

,r_points,r_text,place
0,50,"Dear friends, Back again after a short while t...",Vino Cultura
1,50,"Dear friends, I am back. I have not visited th...",Vino Cultura
2,40,Great food to share and an extensive wine list...,Vino Cultura
3,50,"Love this place, really delicious wine and gre...",Vino Cultura
4,40,It has been several years since I have visited...,Vino Cultura
...,...,...,...
217,50,The brasserie Au Bon Plaisir has the same qual...,Brasserie Au Bon Plaisir
218,10,"My husband took me here for our anniversary, a...",Brasserie Au Bon Plaisir
219,10,I have travelled all over the world and have b...,Brasserie Au Bon Plaisir
220,50,I didn't expect that it will be sooo great the...,Brasserie Au Bon Plaisir


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].astype(str).map(preprocess)
processed_reviews

0      [dear, friends, short, time, world, steak, del...
1      [dear, friends, visited, place, years, kicking...
2      [great, food, share, extensive, wine, list, ma...
3      [love, place, delicious, wine, great, tapas, h...
4      [years, visited, place, saturday, night, busy,...
                             ...                        
217    [brasserie, plaisir, quality, atmosphere, home...
218    [husband, took, anniversary, unfortunately, te...
219    [travelled, world, french, restaurants, place,...
220    [expect, sooo, great, food, partner, went, ann...
221    [local, expat, friends, like, evenings, reserv...
Name: r_text, Length: 558, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=600,
                                           passes=6,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 36,
                                           eval_every = None)

In [17]:
from pprint import pprint

In [18]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"love" + 0.019*"place" + 0.018*"tapas" + 0.015*"wine" + 0.011*"good" '
  '+ 0.008*"cheese" + 0.008*"restaurant" + 0.007*"service" + 0.007*"expensive" '
  '+ 0.006*"ideal"'),
 (1,
  '0.045*"wine" + 0.030*"good" + 0.028*"place" + 0.027*"food" + 0.018*"great" '
  '+ 0.014*"service" + 0.014*"wines" + 0.012*"nice" + 0.011*"french" + '
  '0.010*"atmosphere"'),
 (2,
  '0.016*"wine" + 0.011*"place" + 0.011*"good" + 0.010*"restaurant" + '
  '0.009*"time" + 0.009*"tapas" + 0.009*"night" + 0.008*"food" + 0.007*"great" '
  '+ 0.006*"prices"'),
 (3,
  '0.018*"hotel" + 0.015*"room" + 0.015*"food" + 0.011*"business" + '
  '0.011*"good" + 0.011*"staff" + 0.011*"breakfast" + 0.010*"rooms" + '
  '0.010*"nicosia" + 0.009*"service"'),
 (4,
  '0.016*"place" + 0.012*"food" + 0.010*"nice" + 0.009*"great" + 0.009*"good" '
  '+ 0.008*"wines" + 0.008*"staff" + 0.008*"selection" + 0.007*"friendly" + '
  '0.006*"restaurant"')]


In [19]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.101236  0.015713       1        1  44.035606
3      0.041931  0.097714       2        1  23.865305
2     -0.029414 -0.064501       3        1  14.161634
0      0.076718 -0.058973       4        1   9.796529
4      0.012000  0.010048       5        1   8.140927, topic_info=         Term        Freq       Total Category  logprob  loglift
42       love   59.000000   59.000000  Default  30.0000  30.0000
1427    hotel   85.000000   85.000000  Default  29.0000  29.0000
43      tapas  126.000000  126.000000  Default  28.0000  28.0000
17      place  347.000000  347.000000  Default  27.0000  27.0000
1432     room   75.000000   75.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
2014   french    6.744780  146.356201   Topic5  -5.3834  -0.5690
20    service    6.694603  177.782471   Topic5  -5.3908  -0.7710
43      tapas    5.963134  126.999763   Topic5  -5.5065  -0.5503
1432     room    5.244154   75.925667   Topic5  -5.6350  -0.1644
36       wine    7.164566  463.492554   Topic5  -5.3230  -1.6614

[371 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1401      1  0.903378    account
1401      2  0.064527    account
563       5  0.470147  admitting
2631      3  0.843075       aged
1328      3  0.842607      ahead
...     ...       ...        ...
55        4  0.056845      wines
55        5  0.085268      wines
760       4  0.461268      worse
2139      1  0.202796      young
2139      3  0.608387      young

[737 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1, 5])